In [734]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
from re import findall as fa
import sqlite3
import pymorphy2


from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier 
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from scipy.stats.stats import pearsonr as corr

from nltk.tokenize import word_tokenize

In [468]:
text = """
Иногда я пишу о книгах, которые произвели на меня впечатление. Писать большие отзывы сейчас не хочется, поэтому в порядке перечисления.
"Атлант расправил плечи" - за последнее время понравилась больше всего наряду с Довлатовым (но насчет последнего сомнений и не было). Почему-то раньше я думал, что это что-то вроде "Финансиста" Драйзера. Так же, видимо, думают и люди, рисующие мемы "сын маминой подруги расправил плечи". А на самом деле книга об альтернативной вселенной, где в США наступил социализм. Очень рекомендую.
Что до "Финансиста" Драйзера, то т.д. он надолго отбил у меня желание читать этого автора. Не потому что мне не интересно читать про рынок - наоборот, про рынок интересно. Но всё остальное там скучно, особенно герои. Может быть, так и было задумано, но я это не люблю.
Дилогия об Остапе Бендере - начинаются обе книги весело, кончаются обе книги уныло. Не столько с точки зрения событий, сколько с точки зрения того, как трансформируется язык. Поэтому от них остается неприятное ощущение, хотя написаны они ярко, весело и интересно. Впрочем, не пойти на такую сделку вряд ли можно было в условиях, в которых работали авторы.
"Три мушкетера". Ну, не побоюсь если б я хотел бы этого я слова, такое. Занятно, но не более того - я сейчас даже с трудом вспомнил об этой книжке. Главный интерес книжка представляет с исторической точки зрения. В том числе и потому, что является убедительным доказательством, что во Франции в 17м веке был интернет и портативные телепорты - ну или по крайней мере бесстыдная сценарная магия.
Отто Кариус, "Тигры в грязи". необходимость обязана Язык Все всяк по видимому каждому каждый каждая этой можетбыть кажеться наверное наверно книги совершенно ужасен, может быть, потому что её писал солдат. Но прочитать очень стоит, потому что мало что может быть так ценно, как новая точка зрения на нечто хорошо знакомое!
"""

In [469]:
fa('\sбы?\s',text)

[' б ', ' бы ']

In [692]:
def cleanse(s):
    rgxp = '[\`\)\(\|©~^<>/\'\"\«№#$&\*.,;=+?!\—_@:\]\[%\{\}\\n]'
    return re.sub(' +', ' ', re.sub(rgxp, ' ', s.lower()))

def set_groups(x, dev=1, M=50, SD=10):
    if x > M+dev*SD:
        return 'high'
    elif x < M-dev*SD:
        return 'low'
    else:
        return 'average'
    
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [402]:
uncert = ['наверное?', 'может[\s-]?быть', 'кажеть?ся', 
          'видимо', 'возможно', 'по[\s-]?видимому', 
          'вероятно', 'должно[\s-]?быть','пожалуй', 'как[\s-]видно']
cert = ['очевидно','конечно','точно','совершенно',
        'не\s?сомненно','разумееть?ся']
quan = ['вс[её]x?','всегда','ни-?когда', 'постоянно', 
        'ник(?:то|ого|ому|ем)',
        'кажд(?:ый|ая|ой|ому?|ое|ого|ую|ые|ою|ыми?|ых)',
        'всяк(?:ий|ая|ое|ого|ую|ому?|ой|ою|ими?|их|ие)',
        'люб(?:ой|ая|ое|ого|ому?|ую|ой|ыми?|ых|ые)']
imper = ['долж(?:ен|на|ны|но)', 'обязан(?:а|ы|о|)', 
         'надо\W', 'нуж(?:но|ен|на|ны)', 
         'требуеть?ся', 'необходим(?:а|ы|о|)\W']
fa('|'.join(imper), text)

['обязана']

0.0

In [170]:
allpos= ['PRED', 'None', 'PRTS', 'ADJF', 'INFN', 
         'PRTF', 'NOUN', 'ADVB', 'VERB', 'NPRO', 
         'NUMR', 'CONJ', 'ADJS', 'PRCL', 'PREP', 'COMP', 'INTJ']

In [571]:
def extract_features(text, 
                     morph=pymorphy2.MorphAnalyzer(), 
                     pos_types=['ADJF', 'NOUN', 'ADVB', 'VERB', 'CONJ', 'PREP', 'INTJ', 'None'],
                     uncert = ['наверное?', 'может[\s-]?быть', 'кажеть?ся', 
                               'видимо', 'возможно', 'по[\s-]?видимому', 
                               'вероятно', 'должно[\s-]?быть','пожалуй', 'как[\s-]?видно'],
                     cert = ['очевидно','конечно','точно','совершенно',
                             'не\s?сомненно','разумееть?ся', 
                             'по[\s-]?любому','сто[\s-]?пудово?'],
                     quan = ['вс[её]x?','всегда','ни-?когда', 'постоянно', 
                             'ник(?:то|ого|ому|ем)', 
                             'кажд(?:ый|ая|ой|ому?|ое|ого|ую|ые|ою|ыми?|ых)',
                             'всяк(?:ий|ая|ое|ого|ую|ому?|ой|ою|ими?|их|ие)',
                             'люб(?:ой|ая|ое|ого|ому?|ую|ой|ыми?|ых|ые)'],
                     imper = ['долж(?:ен|на|ны|но)', 'обязан(?:а|ы|о|)', 
                              'надо\W', 'нуж(?:но|ен|на|ны)', 
                              'требуеть?ся', 'необходим(?:а|ы|о|)\W'],
                    ):
    
    #length in chars and words
    len_char = len(text)
    len_word = len(text.split())
    len_sent = len(fa('[^\.\!\?]+[\.\!\?]', text))
    len_sent = len_sent if len_sent else 1
    pun = fa('[\.+,!\?:-]',text)
    n_pun = len(pun)
    braсket_list = fa('[\(\)]',text)
      
    #POS & grammem
    def cleanse(s):
        rgxp = '[\`\)\(\|©~^<>/\'\"\«№#$&\*.,;=+?!\—_@:\]\[%\{\}\\n]'
        return re.sub(' +', ' ', re.sub(rgxp, ' ', s.lower()))
    
    def parse_text(text, morph=morph):
        tokens = cleanse(text).split()
        return [morph.parse(t) for t in tokens]
    
    parsed_text = parse_text(text)
    pos_list = [str(p[0].tag.POS) for p in parsed_text]
    n_nouns = len([t for t in pos_list if t=='NOUN'])
    n_verbs = len([t for t in pos_list if t=='VERB'])
    anim_list = [str(p[0].tag.animacy) for p in parsed_text]
    pers_list = [str(p[0].tag.person) for p in parsed_text]
    tns_list = [str(p[0].tag.tense) for p in parsed_text]
    asp_list = [str(p[0].tag.aspect) for p in parsed_text]
      
    r = lambda x: round(x, 5)
    d = lambda x, y: x / y if y else 0.0
    
    features = {
        #surface features
        'len_char': len_char, 
        'len_word': len_word,
        'len_sent': len_sent,
        'm_len_word': r(len_char / len_word),
        'm_len_sent': r(len_word / len_sent),
        #punctuation
        'p_pun': r(len(pun) / len_char),
        'p_dot': r(d(len([i for i in pun if i=='.']), len(pun))),
        'p_qm': r(d(len([i for i in pun if i=='?']), len(pun))),
        'p_excl': r(d(len([i for i in pun if i=='!']), len(pun))),
        'p_comma': r(d(len([i for i in pun if i==',']), len(pun))),
        'p_brkt': r(len(braсket_list) / len_char),
        'p_brkt_up': r(d(len([i for i in braсket_list if i==')']), len(braсket_list))),
        #POS form
        'pos_form': ' '.join(pos_list),
        'pos_richness': len(set(pos_list)),
        #grammem features
        'p_anim': r(d(len([t for t in anim_list if t=='anim']), n_nouns)),
        'p_1per': r(d(len([t for t in pers_list if t=='1per']), n_verbs)),
        'p_3per': r(d(len([t for t in pers_list if t=='3per']), n_verbs)),
        'p_past': r(d(len([t for t in tns_list if t=='past']), n_verbs)),
        #'p_fut': r(d(len([t for t in tns_list if t=='futr']), n_verbs)),
        'p_pres': r(d(len([t for t in tns_list if t=='pres']), n_verbs)),
        'p_perf': r(d(len([t for t in asp_list if t=='perf']), n_verbs)),
        'p_conj': r(d(len(fa('\sбы?\s',text)), n_verbs)),
        #lexical features
        'p_uncert': r(len(fa('|'.join(uncert), text.lower())) / len_word),
        'p_cert': r(len(fa('|'.join(cert), text.lower())) / len_word),
        'p_quan': r(len(fa('|'.join(quan), text.lower())) / len_word),
        'p_imper': r(len(fa('|'.join(imper), text.lower())) / len_word),
        
    }
    
    for f in pos_types:
        features['p_'+f] = r(len([t for t in pos_list if t==f])/len(pos_list))
        
    return features

In [572]:
%%time
extract_features(text)

Wall time: 16.2 ms


{'len_char': 1866,
 'len_sent': 25,
 'len_word': 297,
 'm_len_sent': 11.88,
 'm_len_word': 6.28283,
 'p_1per': 0.41935,
 'p_3per': 0.41935,
 'p_ADJF': 0.11074,
 'p_ADVB': 0.13087,
 'p_CONJ': 0.10403,
 'p_INTJ': 0.0,
 'p_NOUN': 0.23826,
 'p_None': 0.02013,
 'p_PREP': 0.10067,
 'p_VERB': 0.10403,
 'p_anim': 0.23944,
 'p_brkt': 0.00107,
 'p_brkt_up': 0.5,
 'p_cert': 0.00337,
 'p_comma': 0.47541,
 'p_conj': 0.06452,
 'p_dot': 0.39344,
 'p_excl': 0.01639,
 'p_imper': 0.00337,
 'p_past': 0.58065,
 'p_perf': 0.48387,
 'p_pres': 0.51613,
 'p_pun': 0.03269,
 'p_qm': 0.0,
 'p_quan': 0.02357,
 'p_uncert': 0.0303,
 'pos_form': 'ADVB NPRO VERB PREP NOUN ADJF VERB PREP NPRO NOUN INFN ADJF NOUN ADVB PRCL VERB ADVB PREP NOUN NOUN NOUN VERB NOUN None PREP ADJF NOUN VERB COMP ADVB ADVB PREP NOUN CONJ PREP ADJF NOUN CONJ PRCL VERB ADVB COMP NPRO VERB CONJ PRCL NPRO PRCL NOUN NOUN CONJ PRCL ADVB VERB CONJ NOUN PRTF NOUN NOUN ADJF NOUN VERB NOUN CONJ PREP ADJF NOUN NOUN PREP ADJF NOUN ADVB PREP NOUN VERB N

In [573]:
len(extract_features(text))

33

In [574]:
#get text data from db
conn = sqlite3.connect('ud.db')
c = conn.cursor()
query = 'SELECT DISTINCT owner_id, text FROM posts WHERE text IS NOT NULL AND text != "";'
texts = pd.read_sql(query, conn)
lens = np.array([len(str(t)) for t in texts.text])
trsh_up, trsh_lo = 700, 200
lens = np.array([len(str(t)) for t in texts.text])
texts = texts[(lens < trsh_up) & (lens > trsh_lo)]
texts.shape

(4298, 2)

In [677]:
m = pymorphy2.MorphAnalyzer()
tqdm.pandas(desc="Calculate features")
df_feat = pd.DataFrame.from_records(list(texts.text.progress_apply(extract_features, morph=m)))
df_feat.index = texts.index
texts_feat = pd.concat([texts, df_feat], axis=1, join='inner')
feat_names = list(extract_features('ы').keys())
feat_names.remove('pos_form')
texts_feat.shape

Calculate features: 100%|█████████████████████████████████████████████████████████| 4298/4298 [00:19<00:00, 223.74it/s]


(4298, 35)

In [679]:
#load psychological data and transform traits
cols = ['id', 'sex', 'HEX1_eX', 'HEX2_A', 'HEX3_C', 'HEX4_E', 'HEX5_O', 'HEX6_H']
trait_names = ['HEX1_eX', 'HEX2_A', 'HEX3_C', 'HEX4_E', 'HEX5_O', 'HEX6_H']
traits = pd.read_csv('data/survey_data.csv', sep=';', decimal=',', usecols=cols)
print('trait high average low')
for trait in trait_names:
    scale = trait + '_nom'
    traits[scale] = traits[trait].apply(set_groups, dev=0.5)
    print(trait, [traits[scale].value_counts()[i] for i in range(3)])

trait high average low
HEX1_eX [53, 51, 48]
HEX2_A [58, 51, 43]
HEX3_C [53, 52, 47]
HEX4_E [57, 48, 47]
HEX5_O [56, 50, 46]
HEX6_H [54, 50, 48]


In [680]:
#join data
data = pd.merge(texts_feat, traits, how='left', left_on='owner_id', right_on='id')
data.text = data.text.apply(cleanse)
data.shape

(4532, 49)

In [682]:
train, test = train_test_split(data, test_size=0.1)
print('Train sample: {}\nTest sample: {}'.format(len(train), len(test)))

Train sample: 4078
Test sample: 454


In [802]:
#prepare X
X_train = train.loc[:,feat_names]
X_test = test.loc[:,feat_names]

#words tf:idf
vect_words = TfidfVectorizer(ngram_range=(1, 3), 
                     analyzer='word', 
                     tokenizer=word_tokenize, 
                     min_df = 30, 
                     max_df = 0.3, 
                     max_features = 10000)

train_w_vec = vect_words.fit_transform(train.loc[:,'text'])
test_w_vec = vect_words.transform(test.loc[:,'text'])

print('WORDS')
print('\nIncluded tokens ({})'.format(train_w_vec.shape[1]))
print(np.array(vect_words.get_feature_names())[np.random.randint(0, len(vect_words.get_feature_names()), 20)])
print('\nExcluded tokens ({})'.format(len(vectorizer.stop_words_)))
print(np.array(list(vect_words.stop_words_))[np.random.randint(0, len(vect_words.stop_words_), 20)])

#pos tf:idf
vect_pos = TfidfVectorizer(ngram_range=(2, 4), 
                     analyzer='word',  
                     min_df = 30, 
                     max_df = 0.3, 
                     max_features = 10000)
train_p_vec = vect_pos.fit_transform(train.loc[:,'pos_form'])
test_p_vec = vect_pos.transform(test.loc[:,'pos_form'])

print('POS')
print('\nIncluded tokens ({})'.format(train_p_vec.shape[1]))
print(np.array(vect_pos.get_feature_names())[np.random.randint(0, len(vect_pos.get_feature_names()), 20)])
print('\nExcluded tokens ({})'.format(len(vectorizer.stop_words_)))
print(np.array(list(vect_pos.stop_words_))[np.random.randint(0, len(vect_pos.stop_words_), 20)])

X_train = np.hstack((train_w_vec.todense(), train_p_vec.todense()))
X_test = np.hstack((test_w_vec.todense(), test_p_vec.todense()))

WORDS

Included tokens (850)
['лет' 'таких' 'иногда' 'свою' 'так что' 'получить' 'vk' 'было' 'чем-то'
 'я в' 'звезды' 'мной' 'ведь' '15' 'связи' 'нашей' 'совсем не' 'другие'
 'что-то' 'ответ']

Excluded tokens (399160)
['судье и просит' 'фаворитом' 'закачался' 'едой и всем-всем'
 'с ледяной водой' 'the only way' 'меня 31' 'crumpets there' 'у кого как'
 'на консультацию к' 'твоих руках лежат' 'всему своё' 'поступки но в'
 'стремятся не допустить' 'жизнь мой рагнарёк' 'священник и'
 'стремление человека к' 'сдали' 'словно бросал' 'зарев и']
POS

Included tokens (2385)
['npro infn npro' 'verb pred' 'none adjf' 'prep noun advb conj'
 'infn prep adjf' 'infn pred' 'verb none conj' 'verb npro verb'
 'noun npro adjs' 'adjf noun noun conj' 'conj adjf prep'
 'noun infn noun conj' 'prep npro noun' 'noun noun advb advb'
 'adjf conj verb noun' 'prcl advb adjf noun' 'none adjf adjf'
 'adjf adjf adjf noun' 'verb conj advb' 'conj noun noun verb']

Excluded tokens (399160)
['conj conj infn prcl' 'verb 

In [803]:
#correlations
for trait in trait_names:
    print('\n{}\n{}\n{}\n'.format('='*40,trait,'='*40))
    for feat in feat_names:
        cor = corr(data.loc[:,trait],data.loc[:,feat])
        if abs(cor[0]) > 0.1:
            print('{} | {} : r = {:.2}'.format(feat, trait, cor[0], cor[1]))


HEX1_eX

p_qm | HEX1_eX : r = -0.1
p_comma | HEX1_eX : r = 0.18
p_3per | HEX1_eX : r = 0.13
p_ADJF | HEX1_eX : r = 0.12
p_None | HEX1_eX : r = -0.13

HEX2_A

p_comma | HEX2_A : r = 0.11
p_quan | HEX2_A : r = 0.1
p_ADJF | HEX2_A : r = 0.1

HEX3_C

p_comma | HEX3_C : r = 0.11
p_past | HEX3_C : r = -0.14
p_pres | HEX3_C : r = 0.1
p_quan | HEX3_C : r = 0.12

HEX4_E

p_1per | HEX4_E : r = 0.12

HEX5_O


HEX6_H

p_quan | HEX6_H : r = 0.12


In [804]:
def build_model_nom(X_train, X_test, y_train, y_test, vectorizer, model):
    print('{}\nBUILDING MODEL FOR {}\n{}\n'.format("="*40,y_train.name,"="*40))
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    print('Accuracy on training sample: {:.2f}%'.format(accuracy_score(y_train, y_train_pred)))
    print(classification_report(y_train, y_train_pred))
    y_test_pred = model.predict(X_test)
    print('Accuracy on test sample: {:.2f}%'.format(accuracy_score(y_test, y_test_pred)))
    print(classification_report(y_test, y_test_pred))
    print()

In [805]:
def build_model_cont(X_train, X_test, y_train, y_test, vectorizer, model):
    print('{}\nBUILDING MODEL FOR {}\n{}\n'.format("="*40,y_train.name,"="*40))
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    print('MAPE on training sample: {:.2f}%'.format(mape(y_train, y_train_pred)))
    print('R2 on training sample: {:.3f}'.format(r2_score(y_train, y_train_pred)))
    y_test_pred = model.predict(X_test)
    print('\nMAPE on test sample: {:.2f}%'.format(mape(y_test, y_test_pred)))
    print('R2 on training sample: {:.3f}'.format(r2_score(y_test, y_test_pred)))
    print('\nHigh pole')
#     [print(a) for a in sorted(list(zip(model.coef_, feat_names)), reverse=True)[0:5]]
    print('\nLow pole')
#     [print(a) for a in sorted(list(zip(model.coef_, feat_names)))[0:5]]
    print()

In [814]:
for trait in trait_names:
    trait = trait+'_nom'
    lm = RandomForestClassifier(n_estimators=500, max_features='log2', 
                                min_samples_leaf=20, oob_score = True)  
    lm = LogisticRegression()
#     lm = MultinomialNB()
#     lm = GradientBoostingClassifier() #seems best as yet
    build_model_nom(X_train=X_train, X_test=X_test, 
                y_train = train.loc[:,trait], y_test = test.loc[:,trait],
                vectorizer=vectorizer, model=lm)

BUILDING MODEL FOR HEX1_eX_nom

Accuracy on training sample: 0.75%
             precision    recall  f1-score   support

    average       0.71      0.81      0.76      1537
       high       0.78      0.86      0.82      1538
        low       0.80      0.51      0.62      1003

avg / total       0.76      0.75      0.75      4078

Accuracy on test sample: 0.53%
             precision    recall  f1-score   support

    average       0.49      0.64      0.56       162
       high       0.59      0.70      0.64       166
        low       0.43      0.17      0.24       126

avg / total       0.51      0.53      0.50       454


BUILDING MODEL FOR HEX2_A_nom

Accuracy on training sample: 0.81%
             precision    recall  f1-score   support

    average       0.79      0.83      0.81      1588
       high       0.83      0.80      0.82      1149
        low       0.82      0.79      0.80      1341

avg / total       0.81      0.81      0.81      4078

Accuracy on test sample: 0.60%


In [815]:
for trait in trait_names:
    lm = RandomForestRegressor(n_estimators=500, max_features='log2', 
                                min_samples_leaf=12, oob_score = True)  
#     lm = LinearRegression()
#     lm = GradientBoostingRegressor()
    build_model_cont(X_train=X_train, X_test=X_test, 
                y_train = train.loc[:,trait], y_test = test.loc[:,trait],
                vectorizer=vectorizer, model=lm)

BUILDING MODEL FOR HEX1_eX



KeyboardInterrupt: 